In [8]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime

In [9]:
# JSON 파일 경로
json_file_path = 'C:\\Users\\Administrator\\Downloads\\University\\Graduation project\\Dataset\\yelp_academic_dataset_review.json'

# CSV 파일 경로
csv_file_path = 'checkin.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱
            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            #print(date_time_str, date_time_obj)
            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")

checkin_data 생성 완료.


In [10]:

"""
데이터 개수(row 수) 확인
"""

# CSV 파일 경로
csv_file_path = 'checkin.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 6990280


In [11]:
# CSV 파일 경로
input_file = 'checkin.csv'
output_file = 'checkin_data_Yelp.txt'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    #data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx

# business_id 기준으로 데이터를 정렬
data.sort(key=lambda x: x[1])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.
print(data[:3])

# business_id 정수형으로 변환
idx = 0
before_business_id = data[0][1]
for i in data:
  if i[1] == before_business_id:
    i[1] = idx
  else:
    idx += 1
    before_business_id = i[1]
    i[1] = idx

print(data[:3])

# 다시 유저 번호 기준으로 정렬
data.sort(key=lambda x: x[0])
print(data[:3])

# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

In [ ]:
import pandas as pd

# 파일로부터 데이터를 읽기
# 이 코드에서는 파일 경로 'checkin_data.txt'로 가정합니다.
df = pd.read_csv('checkin_data_Yelp.txt', header=None, names=['ID', 'Value'])

# 고유한 ID를 정렬된 리스트로 추출합니다.
unique_ids = sorted(df['ID'].unique())

# 고유 ID에 대해 0부터 시작하는 새로운 인덱스를 매핑합니다.
id_mapping = {id: index for index, id in enumerate(unique_ids)}

# 이 매핑을 사용하여 모든 ID 값을 업데이트합니다.
df['ID'] = df['ID'].map(id_mapping)

# 결과 확인
print(df)

# 결과를 파일로 저장할 수 있습니다.
df.to_csv('reindexed_data.txt', index=False, header=False)


In [ ]:
import pandas as pd

# 파일로부터 데이터를 읽기
# 이 코드에서는 파일 경로 'checkin_data.txt'로 가정합니다.
df = pd.read_csv('reindexed_data.txt', header=None, names=['ID', 'Value'])

# 고유한 ID를 정렬된 리스트로 추출합니다.
unique_ids = sorted(df['Value'].unique())

# 고유 ID에 대해 0부터 시작하는 새로운 인덱스를 매핑합니다.
id_mapping = {id: index for index, id in enumerate(unique_ids)}

# 이 매핑을 사용하여 모든 ID 값을 업데이트합니다.
df['Value'] = df['Value'].map(id_mapping)

# 결과 확인
print(df)

# 결과를 파일로 저장할 수 있습니다.
df.to_csv('pos.txt', index=False, header=False)


In [ ]:
import pandas as pd
import numpy as np

# 데이터 파일 로드
df = pd.read_csv('pos.txt', header=None, names=['user', 'item'])

# 각 사용자가 가진 아이템의 집합을 만듭니다.
user_items = df.groupby('user')['item'].apply(set)

# 전체 아이템 집합
all_items = set(df['item'])

# 각 사용자별로 가지고 있지 않은 아이템을 찾아 랜덤하게 100개 선택
results = {}
for user, items in user_items.items():
    not_owned_items = list(all_items - items)
    if len(not_owned_items) > 10:
        selected_items = np.random.choice(not_owned_items, 10, replace=False)
    else:
        selected_items = not_owned_items
    results[user] = selected_items

# 결과를 파일로 저장, 사용자 ID 없이 아이템만 저장
with open('neg.txt', 'w') as file:
    for items in results.values():
        items_str = ','.join(map(str, items))
        file.write(f"{items_str}\n")